<a href="https://colab.research.google.com/github/giacomoarienti/data-intensive-lab/blob/master/Recommendation_con_NumPy_nosol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio: Recommendation con Algebra Lineare e NumPy

**Programmazione di Applicazioni Data Intensive**  
Laurea in Ingegneria e Scienze Informatiche  
DISI - Università di Bologna, Cesena

Proff. Gianluca Moro, Roberto Pasolini  
nome.cognome@unibo.it

## Introduzione a Jupyter/Colab

Questo è un file notebook Jupyter (`.ipynb`), composto da una sequenza di:

- celle contenenti testo formattato, come questa
- celle contenenti codice Python eseguibile

Questo è un esempio di cella di codice:

In [ ]:
20 + 20 + 2

42

Le celle di codice possono essere eseguite una alla volta: eseguendo una cella viene riportato sotto la stessa il risultato dell'espressione inserita e/o qualsiasi output ottenuto da `print` o simili.

Cliccare sulla cella e premere **Maiusc + Invio** per eseguirla: il risultato dell'espressione comparirà sotto.

Tutte le celle di codice all'interno di un notebook possono essere modificate e rieseguite liberamente.

_**Importante!** Al contrario di quanto accade nei fogli di calcolo, in Jupyter modificando e rieseguendo una cella **NON** vengono automaticamente aggiornati i risultati delle altre! Ad esempio, se nella prima cella di un notebook si cambia il valore di una variabile da cui dipendono le celle sotto, queste dovranno essere rieseguite esplicitamente per aggiornarne i risultati. Per questo i menu in alto contengono comandi per eseguire in sequenza tutte le celle sopra o sotto quella selezionata._

Si può aggiungere una nuova cella di codice sotto a quella corrente
  - cliccando sul pulsante "+ Code" che appare tra le celle in Colab
  - cliccando sul pulsante "+" in alto in Jupyter

### Comandi principali da tastiera

- **Ctrl + Invio**: esegui cella corrente
- **Maiusc + Invio**: esegui cella corrente e seleziona la successiva
- **Esc**: termina modifica (senza eseguire)

I comandi sotto funzionano sulla cella selezionata solo se non se ne sta modificando il contenuto:
- **Invio**: modifica contenuto
- **A/B**: crea nuova cella sopra/sotto
- **Ctrl+M** seguito da **D**: elimina cella (due volte D in Jupyter)

### Autocompletamento e inline help

Nelle celle di codice sono forniti suggerimenti sul completamento di un nome di variabile, funzione, ecc.
- si provi ad es. a creare una cella di codice (tasto B), e digitare "so"
- attivare l'autocompletamento con Ctrl+Invio su Colab o con Tab su Jupyter
- l'autocompletamento suggerisce ad es. la funzione `sorted`

Per ottenere informazioni in linea su una funzione o metodo `foo`, eseguire una cella con `foo?`
- in alternativa, usare la funzione `help` di Python: `help(foo)`

In [ ]:
len?

In [ ]:
list.append?

## Test librerie

NumPy si trova preinstallato su Colab e Anaconda. Eseguire la seguente cella per verificare il corretto funzionamento di NumPy.

In [ ]:
import numpy as np
np.array([2, 6, 10, 24]).sum()

42

- In caso di errore, verificare di stare utilizzando il kernel corretto e che NumPy sia installato in esso
- Per installare NumPy all'interno dell'ambiente virtuale corrente (può dare errore per permessi insufficienti in caso non si stia utilizzando un ambiente virtuale):

In [ ]:
# rimuovere il "#" nella riga sotto per abilitare il comando
#pip install numpy

## Recommendation: Prevedere le propensioni di acquisto dei clienti

Ogni azienda ha i dati storici di acquisto di ciascun cliente/utente.

Vogliamo **raccomandare/suggerire ai singoli utenti quali prodotti acquistare**, proponendo a ciascuno i prodotti acquistati da altri utenti che in precedenza hanno effettuato acquisti comuni ad esso.

Vediamo come risolvere questo problema utilizzando i concetti dell'**algebra lineare**, rappresentando i dati come **vettori e matrici** ed eseguendo operazioni tra essi.

Usiamo come esempio un set di dati ricavato da vendite su Amazon.

## Scaricamento file dati

Un archivio ZIP con i file necessari per l'esercitazione si trova all'URL https://git.io/fhxQh

Come passaggi preliminari per lavorare sul problema dovremmo:
- verificare i dati sono già presenti nella directory di lavoro;
- se non lo sono, scaricare il file ZIP che li contiene ed estrarne i file nella cartella corrente.

La libreria standard di Python fornisce funzioni per eseguire agevolmente tutte queste operazioni. Eseguire la cella sotto per ottenere i file da utilizzare per l'esercitazione.

In [ ]:
# importo moduli e funzioni necessarie dalla libreria standard
import os.path
from urllib.request import urlretrieve
from zipfile import ZipFile

# se il file "purchases_data.zip" non esiste
if not os.path.exists("purchases_data.zip"):
    # scarica il file dall'URL indicato
    urlretrieve("https://git.io/fhxQh", "purchases_data.zip")
    # apri il file zip ed estrai tutto il contenuto nella directory corrente
    with ZipFile("purchases_data.zip") as f:
        f.extractall()

## Caricamento dati

Il file `users.csv` contiene un elenco degli utenti coinvolti nell'analisi, quelli con almeno 30 acquisti nello storico.

È un file CSV (_Comma Separated Values_) contenente una riga per ogni utente nel formato `IdUtente;Nome`.

Possiamo leggere tali file usando il modulo `csv` della libreria standard di Python.

In [ ]:
import csv

Vogliamo creare un dizionario `users` e inserire in esso un elemento per ogni utente, la cui chiave sia l'ID e il cui valore sia il nome.

1. Usiamo il costrutto `with` e la funzione `open` per aprire il file e chiuderlo automaticamente dopo l'utilizzo.
2. All'interno di `with` creiamo un oggetto `csv.reader` che interpreta il file passato come CSV e ne restituisce le righe scomposte in valori.
3. Usiamo una _comprehension_ per definire un dizionario con una coppia chiave-valore per ogni riga letta dal file, usando `int` per convertire gli ID da stringhe a numeri.

In [ ]:
with open("users.csv", "r") as f:                     # 1
    reader = csv.reader(f, delimiter=";")             # 2
    users = {int(uid): name for uid, name in reader}  # 3

In [ ]:
users

{84: 'malachix',
 7661: 'Kevin Alphonso',
 22759: 'tropic_of_criticism',
 27153: 'Joe Owen "Joe"',
 42298: 'Zane',
 44082: 'David Robson',
 48144: 'Alan R. Holyoak',
 52797: 'Mr N Forbes-warren "author of RESURGENCE and ...',
 58135: 'Jacques COULARDEAU "A soul doctor, so to say"',
 62023: 'Martin Turner "book reviewer"',
 63776: 'brent been',
 81819: 'dsrussell "greyhater"',
 82397: 'juno2',
 83632: 'MOVIE_NUT74@HOTMAIL.COM',
 93938: 'laddie5',
 109403: 'Studebaker Hoch, billythemtn@geocities.com',
 110576: 'LUCIEN LESSARD',
 116105: 'Michael J. Berquist',
 116614: 'behet@stud.uni-frankfurt.de',
 125462: 'Alex Udvary',
 125883: 'flickjunkie',
 126922: 'Sean Ares Hirsch',
 130054: 'lecorel@hotmail.com',
 138128: 'zara_azari',
 140844: 'yygsgsdrassil "yygsgsdrassil"',
 158428: 'Vincent Tesi "Vinny"',
 158855: 'NEO-CS- "Cyber Soldier"',
 164121: 'Dustin J. Hetke',
 165618: 'Scott T. Rivers',
 172716: 'Aaron Amos',
 187485: 'jakelamotta',
 188455: 'A. Andersen',
 190242: 'snlkidsinhall',


Il file `items.csv` contiene i prodotti distinti acquistati dagli utenti in formato analogo al file `users.csv`, con righe `IdProdotto;Nome`.

Ne vogliamo salvare il contenuto in un dizionario `items`, ottenuto come fatto sopra con `users`.

In [ ]:
with open("items.csv", "r") as f:
    reader = csv.reader(f, delimiter=";")
    items = {int(iid): name for iid, name in reader}

In [ ]:
items

{20: 'Age of Innocence [VHS]',
 44: 'Detour [VHS]',
 46: 'Strange Love of Martha Ivers [VHS]',
 66: 'Cool Runnings [VHS]',
 73: 'Lost World [VHS]',
 79: 'The Day the Earth Caught Fire [VHS]',
 87: 'Blood & Sand [VHS]',
 115: 'Band of Angels [VHS]',
 142: 'The Autobiography of Miss Jane Pittman [VHS]',
 170: 'Death Wish - The Face Of Death [VHS]',
 173: 'The Longest Day [VHS]',
 174: 'Doctor Who - Robot [VHS]',
 204: 'Doctor Who - City of Death [VHS]',
 228: 'Frances [VHS]',
 236: '5 Fingers [VHS]',
 244: 'Call Northside 777 [VHS]',
 263: 'Heaven & Earth [VHS]',
 264: 'The Egg and I (The Ma & Pa Kettle Collection) [VHS]',
 272: 'Crash Dive [VHS]',
 276: 'Pinky [VHS]',
 286: 'The Seventh Seal [VHS]',
 296: 'Tombstone [VHS]',
 333: "Tim Burton's The Nightmare Before Christmas [VHS]",
 381: 'Drugstore Cowboy [VHS]',
 386: 'Thumbelina [VHS]',
 395: 'My Girl 2 [VHS]',
 440: 'Wicker Man [VHS]',
 448: 'D2 - The Mighty Ducks [VHS]',
 457: 'A Charlie Brown Christmas',
 481: 'Blue [VHS]',
 524: '

Il file CSV `purchases-2000.csv` contiene i dati sugli acquisti effettuati dagli utenti analizzati fino alla fine del 2000: per ciascun acquisto registrato, il file contiene una riga `IdUtente;IdProdotto`.

Usiamo le funzioni viste sopra per leggere il file, creando stavolta un insieme `purchases` di tuple `(uid, iid)`.

In [ ]:
with open("purchases-2000.csv", "r") as f:
    reader = csv.reader(f, delimiter=";")
    purchases_set = set(
        (int(uid), int(iid))
        for uid, iid in reader
    )

In [ ]:
purchases_set

{(641520, 44037),
 (279452, 100933),
 (865352, 96240),
 (390353, 57780),
 (138128, 43885),
 (190242, 51231),
 (1228076, 54695),
 (188455, 99904),
 (365071, 44176),
 (774984, 43771),
 (442793, 57190),
 (1098934, 46016),
 (1228076, 102172),
 (426926, 42913),
 (502896, 44312),
 (889499, 44163),
 (580709, 2302),
 (116105, 95933),
 (362142, 48709),
 (811933, 43447),
 (1092996, 95302),
 (1092996, 57395),
 (889499, 49532),
 (1234283, 5462),
 (63776, 47759),
 (753999, 56561),
 (827287, 96022),
 (1091373, 43911),
 (48144, 43407),
 (63776, 104096),
 (743388, 43824),
 (464319, 55317),
 (533414, 43420),
 (390353, 8101),
 (140844, 42950),
 (999745, 51683),
 (125883, 4572),
 (116105, 7637),
 (365712, 43283),
 (116614, 43206),
 (138128, 44356),
 (502896, 1278),
 (771431, 8853),
 (130054, 43982),
 (164121, 49606),
 (58135, 96450),
 (827287, 3525),
 (964312, 43262),
 (724868, 6873),
 (464319, 103962),
 (791603, 101696),
 (533414, 47971),
 (774984, 5288),
 (1201103, 8009),
 (399035, 50331),
 (218221, 96

## Algebra Lineare e NumPy

Oggetto di studio dell'**algebra lineare** sono i vettori, le matrici e le principali operazioni tra di essi:

- con vettori e matrici si possono rappresentare dati da analizzare,
- informazioni d'interesse si possono estrarre con operazioni come il prodotto tra matrici.

**NumPy** è una libreria Python molto usata per lavorare con vettori, matrici e array N-dimensionali in generale. È alla base di altre librerie, ad es. _pandas_ che utilizzeremo nei prossimi laboratori.

Sugli array NumPy si possono compiere efficientemente operazioni elemento per elemento, aggregazioni, operazioni di algebra lineare, ecc.

Iniziando importando il package `numpy` e assegnandogli l'alias convenzionale `np`.

In [ ]:
import numpy as np

## Rappresentare gli acquisti in forma di matrice

L'insieme `purchases` degli acquisti degli utenti analizzati può essere rappresentato in una matrice **P** binaria, ovvero di valori 0 e 1 dove
- ogni **riga** corrisponde ad un **utente**,
- ogni **colonna** corrisponde ad un **prodotto** distinto,
- il valore della cella _i,j_ è 1 se l'utente i ha acquistato il prodotto j, 0 altrimenti.

$$\mathbf{P} = \begin{bmatrix}0&1&0&1&0&\cdots\\1&0&0&1&0&\cdots\\0&0&1&0&1&\ldots\\\vdots&\vdots&\vdots&\vdots&\vdots&\ddots\end{bmatrix}$$

Vediamo come costruire tale matrice.

### Assegnazione di indici a utenti e prodotti

Iniziamo definendo **a quali utenti e prodotti corrispondano** rispettivamente **righe e colonne** della matrice.

Costruiamo un dizionario `user_indices` che associ ad ognuno degli N ID utente (`uid`) un numero tra 0 e N-1 (`index`).

Usiamo `enumerate` per ottenere tuple `(index, uid)`, dove _index_ è un numero progressivo da 0 a N-1.

`enumerate([X, Y, Z])` $\Rightarrow$ `(0, X), (1, Y), (2, Z)`

Usiamo `sorted` per ottenere le chiavi in ordine crescente _(non è strettamente necessario, ma garantisce la piena riproducibilità dei passaggi)_.

In [ ]:
user_indices = {uid: index for index, uid in enumerate(sorted(users.keys()))}
user_indices

{84: 0,
 7661: 1,
 22759: 2,
 27153: 3,
 42298: 4,
 44082: 5,
 48144: 6,
 52797: 7,
 58135: 8,
 62023: 9,
 63776: 10,
 81819: 11,
 82397: 12,
 83632: 13,
 93938: 14,
 109403: 15,
 110576: 16,
 116105: 17,
 116614: 18,
 125462: 19,
 125883: 20,
 126922: 21,
 130054: 22,
 138128: 23,
 140844: 24,
 158428: 25,
 158855: 26,
 164121: 27,
 165618: 28,
 172716: 29,
 187485: 30,
 188455: 31,
 190242: 32,
 192779: 33,
 218221: 34,
 218789: 35,
 224067: 36,
 229993: 37,
 231204: 38,
 239871: 39,
 260996: 40,
 261279: 41,
 274676: 42,
 279452: 43,
 281883: 44,
 293553: 45,
 296466: 46,
 339981: 47,
 360171: 48,
 362142: 49,
 365071: 50,
 365712: 51,
 368839: 52,
 390353: 53,
 391690: 54,
 393881: 55,
 395248: 56,
 399035: 57,
 424169: 58,
 426926: 59,
 429177: 60,
 431084: 61,
 431136: 62,
 439900: 63,
 442793: 64,
 446291: 65,
 452809: 66,
 458094: 67,
 464319: 68,
 483531: 69,
 493152: 70,
 502896: 71,
 503195: 72,
 508200: 73,
 512280: 74,
 516868: 75,
 529431: 76,
 533414: 77,
 536662: 78,
 5

Eseguiamo un'operazione simile per ottenere un dizionario `item_indices` con la numerazione degli oggetti.

In [ ]:
item_indices = {iid: index for index, iid in enumerate(sorted(items.keys()))}
item_indices

{20: 0,
 44: 1,
 46: 2,
 66: 3,
 73: 4,
 79: 5,
 87: 6,
 115: 7,
 142: 8,
 170: 9,
 173: 10,
 174: 11,
 204: 12,
 228: 13,
 236: 14,
 244: 15,
 263: 16,
 264: 17,
 272: 18,
 276: 19,
 286: 20,
 296: 21,
 333: 22,
 381: 23,
 386: 24,
 395: 25,
 440: 26,
 448: 27,
 457: 28,
 481: 29,
 524: 30,
 555: 31,
 579: 32,
 585: 33,
 586: 34,
 601: 35,
 606: 36,
 612: 37,
 684: 38,
 712: 39,
 715: 40,
 721: 41,
 724: 42,
 738: 43,
 741: 44,
 750: 45,
 773: 46,
 842: 47,
 895: 48,
 903: 49,
 909: 50,
 913: 51,
 915: 52,
 926: 53,
 935: 54,
 958: 55,
 1005: 56,
 1008: 57,
 1033: 58,
 1041: 59,
 1048: 60,
 1051: 61,
 1063: 62,
 1083: 63,
 1086: 64,
 1119: 65,
 1129: 66,
 1139: 67,
 1176: 68,
 1230: 69,
 1235: 70,
 1255: 71,
 1275: 72,
 1278: 73,
 1288: 74,
 1291: 75,
 1307: 76,
 1312: 77,
 1319: 78,
 1320: 79,
 1335: 80,
 1362: 81,
 1368: 82,
 1382: 83,
 1412: 84,
 1430: 85,
 1438: 86,
 1470: 87,
 1484: 88,
 1531: 89,
 1549: 90,
 1573: 91,
 1574: 92,
 1582: 93,
 1600: 94,
 1633: 95,
 1641: 96,
 1643:

Abbiamo così ottenuto dizionari che mappano ID utenti e prodotti a righe e colonne della matrice.

Ad esempio, la riga corrispondente all'utente con ID 63776 ha indice...

In [ ]:
user_indices[63776]

10

### Inizializzazione della matrice

Per allocare la matrice degli acquisti, definiamone dapprima la _forma_, ovvero il numero di righe e di colonne.

Questi sono pari rispettivamente al numero di utenti e di prodotti, che salviamo in due variabili per comodità.

In [ ]:
n_users = len(users)
n_items = len(items)
print(f"{n_users} utenti, {n_items} prodotti")

178 utenti, 3384 prodotti


Creiamo ora la matrice `purchases` inizializzando tutti i valori a 0, ovvero senza alcun acquisto.

Usiamo per questo la funzione `zeros` passando la forma desiderata della matrice.

Dato che la matrice conterrà solo valori interi (0 e 1), specifichiamo `int` come datatype

- `int` verrà convertito ad un dtype con una precisione specifica, es. `int64`

In [ ]:
purchases = np.zeros((n_users, n_items), dtype=int)

### Esercizio 1: Scrittura acquisti nella matrice

Per ottenere la matrice degli acquisti effettiva, impostare a 1 gli elementi corrispondenti alle coppie utente-oggetto nell'insieme `purchases_set`

- usare opportunamente i dizionari `user_indices` e `item_indices` creati sopra
- per impostare un valore nella matrice: `matrice[indice_riga, indice_colonna] = valore`

In [ ]:
for user_id, user_idx in user_indices.items(): # for each user
  for item_id, item_idx in item_indices.items(): # for each item
    if (user_id, item_id) in purchases_set: # if the tuple (user_id, item_id ) exists in the purchases
      purchases[user_idx, item_idx] = 1

Eseguire la cella TEST sotto per verificare la correttezza. Se la condizione data è soddisfatta viene stampato "OK", altrimenti `assert` genera un errore.

In [ ]:
# TEST
assert purchases[0, 0] == 0
assert purchases[-4, 10] == 1
assert purchases[-2, 12] == 1
assert purchases[-1, -1] == 0

Visualizziamo una porzione della matrice ottenuta...

In [ ]:
purchases[-5:, :15]   # ultime 5 righe, prime 15 colonne

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

Vediamo ad esempio che l'utente della penultima riga ha acquistato i prodotti di indice 11 e 12

### Array booleani

Oltre ai comuni array numerici, NumPy supporta array booleani, contenenti come unici valori possibili `False` e `True`

Abbiamo rappresentato `purchases` con valori 0 e 1 perché è una notazione comune, ma può essere rappresentata anche come array booleano.

Usiamo il metodo `astype` per ottenere una copia della matrice convertendone il datatype.

Viene seguita la convenzione di Python per cui 0 diventa `False` e ogni altro numero `True`.

In [ ]:
purchases_bool = purchases.astype(bool)

Otteniamo una matrice con 0 e 1 convertiti rispettivamente in `False` e `True`.

In [ ]:
purchases_bool[-5:, :15]   # ultime 5 righe, prime 15 colonne

array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False,  True, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False,  True,  True, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False]])

Questa versione booleana della matrice ci servirà per alcune operazioni.

## Associare nomi a righe e colonne

I dizionari `*_indices` permettono, dato l'ID di un utente o prodotto, di risalire alla riga o colonna corrispondente della matrice.

Creiamo ora un vettore `user_names` che contenga i nomi degli utenti nelle posizioni corrispondenti alle righe della matrice.

Allochiamo un vettore vuoto con lunghezza pari al numero di utenti e tipo di elementi `object`, ovvero oggetti Python arbitrari

Come forma dell'array indichiamo `n_users` (valore singolo), che equivale a `(n_users, )` (tupla di un elemento, quindi vettore)

In [ ]:
user_names = np.empty(n_users, dtype=object)

### Esercizio 2: Creazione vettori nomi

- **(2a)** Riempire il vettore `user_names` con i nomi degli utenti corrispondenti alle righe di `purchases`
  - usare i dizionari `users` e `user_indices`
- **(2b)** Creare e riempire in modo simile un vettore `item_names` con i nomi degli oggetti corrispondenti alle colonne di `purchases`

In [ ]:
user_names = np.array([users[i] for i in user_indices.keys()])
user_names[:10]

array(['malachix', 'Kevin Alphonso', 'tropic_of_criticism',
       'Joe Owen "Joe"', 'Zane', 'David Robson', 'Alan R. Holyoak',
       'Mr N Forbes-warren "author of RESURGENCE and ...',
       'Jacques COULARDEAU "A soul doctor, so to say"',
       'Martin Turner "book reviewer"'], dtype='<U48')

In [ ]:
item_names  = np.array([items[i] for i in item_indices.keys()])
item_names[:10]

array(['Age of Innocence [VHS]', 'Detour [VHS]',
       'Strange Love of Martha Ivers [VHS]', 'Cool Runnings [VHS]',
       'Lost World [VHS]', 'The Day the Earth Caught Fire [VHS]',
       'Blood & Sand [VHS]', 'Band of Angels [VHS]',
       'The Autobiography of Miss Jane Pittman [VHS]',
       'Death Wish - The Face Of Death [VHS]'], dtype='<U200')

In [ ]:
# TEST
assert user_names[0] == "malachix"
assert item_names[0] == "Age of Innocence [VHS]"

Possiamo accedere a singoli elementi o intervalli dell'array come nelle liste di Python, indicando un indice o un intervallo `inizio:fine` tra parentesi quadre.

Possiamo ad esempio ottenere il nome di un singolo prodotto dato l'indice della colonna.

In [ ]:
item_names[11]

'Doctor Who - Robot [VHS]'

In più, rispetto alle liste di Python, possiamo estrarre molteplici valori dal vettore passando una **lista di indici**.

In [ ]:
item_names[[3112, 1417, 3109]]

array(['Star Wars - Episode IV, A New Hope [VHS]',
       'Star Wars - Episode V, The Empire Strikes Back [VHS]',
       'Star Wars - Episode VI, Return of the Jedi [VHS]'], dtype='<U200')

## Selezione elementi tramite array booleani

Possiamo usare l'**indicizzazione booleana** per estrarre elementi che corrispondano ad una condizione.

Consideriamo ad esempio la riga 176 della matrice, che indica gli acquisti dell'utente...

In [ ]:
user_names[176]

'G.Spider'

Estraiamo la riga corrispondente dalla matrice `purchases_bool` per ottenere un vettore in forma booleana.

Per estrarre uno o più valori da una matrice, usiamo la forma `matrice[righe, colonne]`, dove per `righe` e `colonne` specifichiamo valori singoli, intervalli o liste.

Per ottenere una riga intera al posto di `colonne` indichiamo `:`, che è una forma breve per `0:numero_colonne`.

In [ ]:
purchased_by_user_176 = purchases_bool[176, :]

array([False, False, False, ..., False, False, False])

A sua volta, `[N, :]` si può abbreviare in `[N]`

In [ ]:
purchased_by_user_176 = purchases_bool[176]

Avendo selezionato una singola riga di una matrice (2 dimensioni), l'oggetto che otteniamo è un vettore (1 dimensione).

In [ ]:
# visualizzo i primi 20 valori
purchased_by_user_176[:20]

array([False, False, False, False, False, False, False, False, False,
       False, False,  True,  True, False, False, False, False, False,
       False, False])

Possiamo usare questo vettore booleano come selettore del vettore `item_names` per estrarre i soli valori di `item_names` corrispondenti a valori `True`.

In questo modo otteniamo un vettore con i **nomi** dei soli **prodotti acquistati dall'utente**.

In [ ]:
item_names_purchased_by_user_176 = item_names[purchased_by_user_176]

# visualizzo i primi 10 elementi
item_names_purchased_by_user_176[:10]

array(['Doctor Who - Robot [VHS]', 'Doctor Who - City of Death [VHS]',
       'Doctor Who: The Curse of Peladon [VHS]',
       'Doctor Who:  Time and The Rani [VHS]',
       'Doctor Who - Ghost Light [VHS]',
       'Doctor Who - The Visitation / Black Orchid [VHS]',
       'Doctor Who - Kinda [VHS]',
       'Doctor Who - The Mark of the Rani [VHS]',
       'Doctor Who - The Leisure Hive [VHS]',
       'Star Wars - Episode I, The Phantom Menace [VHS]'], dtype=object)

Si noti che per questa operazione **è necessario usare l'array booleano** come selettore. Usando quello binario si avrebbe un risultato errato, in quanto i numeri 0 e 1 sarebbero interpretati come indici di valori da selezionare più volte all'interno di `item_names`.

In [ ]:
item_names[purchases[176, :]]

array(['Age of Innocence [VHS]', 'Age of Innocence [VHS]',
       'Age of Innocence [VHS]', ..., 'Age of Innocence [VHS]',
       'Age of Innocence [VHS]', 'Age of Innocence [VHS]'], dtype=object)

## Statistiche sugli acquisti

Gli array forniscono metodi per calcolare valori aggregati: somma, media, min/max, ecc.

Ad esempio il metodo `sum` calcola la somma di tutti gli elementi di un array.

Possiamo utilizzarlo per ottenere il numero totale di acquisti analizzati.

In [ ]:
purchases.sum()

9683

Posso ottenere lo stesso risultato effettuando la somma sulla matrice booleana, in quanto `False` e `True` trattati come numeri valgono 0 e 1 rispettivamente.

In generale, `sum` può essere usato per contare quanti sono i valori `True` in un array booleano.

In [ ]:
purchases_bool.sum()

9683

Oltre ad aggregare tutti i valori, possiamo compiere aggregazioni per righe (asse 0) o per colonne (asse 1).

Applichiamo la somma per righe e per colonne alla matrice degli acquisti.

- `sum(1)` -> sommo tra loro le colonne -> ottengo il numero di prodotti acquistati per ogni utente (riga)
- `sum(0)` -> sommo tra loro le righe -> ottengo il numero di utenti che hanno acquistato ciascun prodotto (colonna)

In [ ]:
user_purchases = purchases.sum(1)
item_purchases = purchases.sum(0)

In [ ]:
# numero di acquisti dei primi 10 clienti
user_purchases[:10]

array([38, 32, 50, 41, 34, 47, 72, 40, 33, 51])

In [ ]:
# numero di clienti che hanno acquistato i primi 10 oggetti
item_purchases[:10]

array([1, 2, 1, 4, 1, 1, 1, 1, 1, 2])

Oltre a `sum` si ricordano tra i metodi più comuni:

- `mean` per la media,
- `min` e `max` per ottenere **i valori** minimo e massimo dell'array,
- `argmin` e `argmax` per ottenere **gli indici** all'interno di un vettore del valori minimo e massimo.

Nota: per ognuno di questi metodi, ad es. `X.sum()`, esiste una funzione equivalente `np.sum(X)` che si può applicare anche ad oggetti _array-like_, ad es. le normali `list` di Python.

In [ ]:
# applico np.sum ad un array NumPy
np.sum( np.array([1, 2, 3]) )

6

In [ ]:
# applico np.sum ad una lista Python
np.sum([1, 2, 3])

6

### Esercizio 3: Estrazione informazioni dai vettori

Utilizzando i vettori `user_purchases`, `item_purchases`, `user_names` e `item_names` ricavare:
- **(3a)** il numero di acquisti medio per ogni utente
- **(3b)** il numero di acquisti del prodotto più acquistato
- **(3c)** il nome dell'utente che ha effettuato più acquisti
- **(3d)** il numero di utenti che hanno acquistato almeno 50 prodotti
- **(3e)** i nomi dei prodotti acquistati da almeno 35 utenti

_**Suggerimento:** per risolvere i punti 3d e 3e, usare operatori di confronto (`>`, `>=`, ...) per ricavare array booleani da quelli numerici._

## Similarità tra utenti

Vogliamo suggerire prodotti agli utenti in base a **cos'hanno acquistato utenti simili**.

Per determinare quanto due utenti siano "simili" possiamo contare **quanti sono i prodotti che entrambi hanno acquistato**.

Per far ciò, verifichiamo il numero di elementi nell'**intersezione** degli insiemi $P_u$ (operatore `&`) dei prodotti acquistati da ciascun utente $u$.

$$ S_{u,v} = \lvert P_u \wedge P_v \rvert $$

Avendo un vettore (riga della matrice) booleano per ogni utente, possiamo trovare quali sono gli acquisti comuni tra due utenti tramite l'AND elemento per elemento.

Estraiamo ad esempio gli acquisti comuni tra i primi due utenti nella matrice.

In [ ]:
purchases_bool[0] & purchases_bool[1]

array([False, False, False, ..., False, False, False])

Nella rappresentazione binaria (0 e 1), questo corrisponde alla **moltiplicazione elemento per elemento**, in quanto il prodotto elemento per elemento è 1 solo dove in entrambi i vettori c'è 1, altrimenti è 0.

In [ ]:
purchases[0] * purchases[1]

array([0, 0, 0, ..., 0, 0, 0])

Per sapere quanti sono i prodotti acquistati in comune posso contare i `True` o gli 1 nell'array col metodo `sum`.

In [ ]:
np.sum( purchases[0] * purchases[1] )

4

Il numero di prodotti acquistati in comune è quindi pari alla **somma dei prodotti** tra le corrispondenti righe della matrice.

Questo corrisponde al **prodotto scalare** (_dot product_) tra le due righe, che può essere estratto efficientemente con la funzione/metodo `dot` o l'operatore `@`.

In [ ]:
np.dot(purchases[0], purchases[1])

4

In [ ]:
purchases[0].dot(purchases[1])

4

In [ ]:
purchases[0] @ purchases[1]

4

### Matrice di similarità

Vogliamo ottenere in una matrice `similarity` NxN tutte le similarità reciproche tra gli N utenti, ovvero i prodotti scalari tra tutte le righe di `purchases`.

```
similarity[i, j] == purchases[i, :] @ purchases[j, :]
```

Il **prodotto tra matrici** restituisce una matrice di prodotti scalari tra le righe di una matrice e le colonne di un'altra.

```
(A @ B)[i, j] == A[i, :] @ B[:, j]
```

Se vogliamo calcolare i prodotti tra le righe di A e le righe di B, possiamo calcolare il prodotto tra A e la **trasposta** di B, estraibile con `B.T`.

```
(A @ B.T)[i, j] == A[i, :] @ B[j, :]
```

Per ottenere i prodotti scalari tra tutte le righe di `purchases` possiamo quindi **moltiplicarla per la sua trasposta**.

In [ ]:
similarity = purchases @ purchases.T

Otteniamo una matrice quadrata, di ordine pari al numero di utenti.

In [ ]:
similarity.shape

(178, 178)

Visualizziamo un estratto della matrice...

In [ ]:
similarity[:10, :10]

array([[38,  4,  1,  0,  1,  2,  1,  1,  0,  2],
       [ 4, 32,  1,  0,  0,  2,  0,  1,  2,  4],
       [ 1,  1, 50,  1,  4,  2,  1,  2,  1,  7],
       [ 0,  0,  1, 41,  1,  1,  3,  1,  1,  2],
       [ 1,  0,  4,  1, 34,  1,  0,  2,  1,  2],
       [ 2,  2,  2,  1,  1, 47,  1,  0,  2,  1],
       [ 1,  0,  1,  3,  0,  1, 72,  1,  1,  2],
       [ 1,  1,  2,  1,  2,  0,  1, 40,  1,  6],
       [ 0,  2,  1,  1,  1,  2,  1,  1, 33,  3],
       [ 2,  4,  7,  2,  2,  1,  2,  6,  3, 51]])

Possiamo ad esempio notare che

- tra i primi due utenti ci sono 4 oggetti acquistati in comune (come visto sopra),
- il primo ed il quarto utente non hanno alcun acquisto in comune.

In [ ]:
similarity[0, 1]

4

In [ ]:
similarity[0, 3]

0

Possiamo verificare che la matrice sia simmetrica controllando se è uguale alla sua trasposta.

L'operatore `==` applicato tra matrici restituisce una matrice booleana con il confronto elemento per elemento: usiamo il metodo `all` (o la funzione `np.all`) per verificare che tutti i valori siano `True`.

In [ ]:
(similarity == similarity.T).all()

True

In [ ]:
np.all(similarity == similarity.T)

True

In alternativa NumPy offre una funzione `array_equal` per verificare l'uguaglianza tra due array: questa evita errori in caso si comparino array di dimensioni incompatibili

In [ ]:
np.array_equal(similarity, similarity.T)

True

### Diagonale della matrice

La diagonale della matrice creata contiene i prodotti scalari di ciascuna riga della matrice originale con se stessa.

Si può verificare che sono uguali al numero di acquisti per colonne calcolato sopra.

In [ ]:
np.array_equal(np.diag(similarity), user_purchases)

True

Prima di procedere, **impostiamo a 0 tutti i valori sulla diagonale** per far sì che tra i simili di ciascun utente non sia incluso lui stesso.

La funzione `fill_diagonal` imposta gli elementi della diagonale al valore dato.

In [ ]:
np.fill_diagonal(similarity, 0)
similarity[:5, :5]

array([[0, 4, 1, 0, 1],
       [4, 0, 1, 0, 0],
       [1, 1, 0, 1, 4],
       [0, 0, 1, 0, 1],
       [1, 0, 4, 1, 0]])

### Esercizio 4: Estrazione informazioni dalla matrice similarità

- **(4a)** Qual è il numero massimo di prodotti in comune tra due utenti?
- **(4b)** Qual è il nome dell'utente che ha più acquisti in comune con l'utente con ID 7661?

## Stimare il potenziale interesse nei prodotti

Vogliamo stimare **quanto ciascun utente sia potenzialmente interessato** in ciascun prodotto non ancora acquistato.

Possiamo stimarlo in base a quanto il prodotto **sia stato acquistato da utenti simili**.

Associamo per ogni utente U e prodotto P un _punteggio d'interesse_ pari alla **somma delle similarità degli altri utenti** che hanno acquistato P.

$$ I_{u,i} = \sum_{v \in \mathbf{U}: i \in P_v \wedge u \neq v} S_{u,v} $$

Questa somma equivale in pratica al prodotto scalare tra:

- la riga relativa a U di `similarity` che indica i punteggi di similarità con gli altri utenti e
- la colonna relativa a P di `purchases` che indica quali utenti lo hanno acquistato.

Ad esempio, l'interesse stimato del 1° utente verso il 2° prodotto è:

In [ ]:
similarity[0, :] @ purchases[:, 1]

5

### Esercizio 5: calcolo matrice punteggi d'interesse

- **(5a)** Costruire una matrice `interest` contenente in ciascuna cella di riga _i_ e colonna _j_ l'interesse stimato dell'utente _i_-esimo verso il prodotto _j_-esimo secondo la formula indicata sopra.
- **(5b)** Verificare che la forma di `interest` sia la stessa di quella di `purchases` (utenti x prodotti).
- **(5c)** Impostare a 0 i valori nella matrice `interest` relativi a coppie utente-prodotto _(U, P)_ in cui _U_ ha già acquistato _P_.

In [ ]:
# TEST
assert interest[0, 0] == 0
assert interest[3, 3] == 8
assert interest[-4, 10] == 0
assert interest[-2, 12] == 0
assert interest[-1, -1] == 5
print("OK")

Visualizziamo un estratto della matrice...

In [ ]:
interest[:10, :10]

array([[ 0,  5,  0,  4,  4,  3,  1,  0,  4,  3],
       [ 3,  7,  1,  6,  1,  1,  2,  1,  2,  0],
       [ 6,  9,  0, 12,  0,  0,  1,  0,  9,  0],
       [ 1,  2,  2,  8,  0,  0,  0,  2,  5,  2],
       [ 3,  2,  0,  5,  0,  0,  0,  0,  2,  0],
       [ 2,  5,  0, 10,  0,  0,  1,  0,  5,  0],
       [ 0,  5,  5,  5,  1,  2,  1,  5,  2,  3],
       [ 1,  5,  0,  7,  0,  0,  0,  0,  0,  3],
       [ 2,  4,  0,  0,  0,  0,  1,  0,  4,  4],
       [ 6, 17,  0, 25,  0,  0,  0,  0,  9,  2]])

Ad esempio si ritiene che il 10° utente (ultima riga) sia molto interessato al 4° prodotto

## Ottenere _N_ suggerimenti di acquisto per ogni utente

Da migliaia di prodotti nel catalogo, vogliamo suggerirne **un numero limitato ad ogni utente** massimizzando la probabilità di acquisto.

Fissiamo un numero _N_ di prodotti da suggerire...

In [ ]:
N = 20

Vogliamo selezionare per ogni utente gli **_N_ prodotti con "potenziale interesse" maggiore**.

Possiamo procedere per ciascun utente assegnando un "ranking" a ciascun prodotto in base al punteggio d'interesse.

Una volta fatto questo, selezioniamo per ogni cliente gli _N_ prodotti col ranking migliore.

### Estrarre l'ordine dei valori: il metodo `argsort`

Il metodo `argsort` su un vettore restituisce un vettore con i suoi **indici** (da 0 a N-1) **ordinati** secondo i valori.

In [ ]:
# indici:       0   1   2   3    4    5   6
x = np.array([320, 80, 20, 40, 160, 640, 10])
x.argsort()

array([6, 2, 3, 1, 4, 0, 5])

Significa che l'elemento minimo è quello di indice 6 (10), seguito da quello di indice 2 (20) e così via fino al massimo di indice 5 (640).

In pratica, selezionando gli elementi dell'array nell'ordine dato da `argsort`, otteniamo un array con gli elementi in ordine crescente.

In [ ]:
x[x.argsort()]

array([ 10,  20,  40,  80, 160, 320, 640])

Nel caso di una matrice, l'operazione va eseguita **lungo una dimensione** a scelta (riga per riga o colonna per colonna).

In [ ]:
np.array([[32,  8,  2,  4, 16, 64,  1],
          [ 8, 16,  4, 64, 32,  1,  2]]).argsort(1)   # 1 = riga per riga

array([[6, 2, 3, 1, 4, 0, 5],
       [5, 6, 2, 0, 1, 4, 3]])

### Calcolare il "ranking" dei prodotti

Applicando l'operazione `argsort` due volte otteniamo il "ranking" dei valore, ovvero l'indice che ogni elemento avrebbe nell'array ordinato.

Otteniamo quindi un array che associa **0 all'elemento più basso, 1 al secondo e così via**.

In [ ]:
x

array([320,  80,  20,  40, 160, 640,  10])

In [ ]:
x.argsort()

array([6, 2, 3, 1, 4, 0, 5])

Da leggersi: l'elemento più basso di `x` ha indice 6, il secondo ha indice 2, ...

In [ ]:
x.argsort().argsort()

array([5, 3, 1, 2, 4, 6, 0])

Da leggersi: il primo elemento di `x` è il 5° (contando da 0°) più basso, il secondo è il 3°, ...

_(chi fosse curioso può leggere [questa dimostrazione formale](https://www.berkayantmen.com/rank.html) che spiega perché si ottiene questo risultato)_

Normalmente l'array risultante contiene 0 per l'elemento più basso e N-1 per quello più alto. Per invertire gli indici, restituendo quindi 0 per l'elemento più alto, è possibile applicare la stessa operazione all'array inverso.

In [ ]:
(-x).argsort().argsort()

array([1, 3, 5, 4, 2, 0, 6])

Da leggersi: il primo elemento di `x` è il 1° (contando da 0°) più alto, il secondo è il 3°, ...

### Esercizio 6: Selezionare i prodotti da suggerire

- **(6a)** Estrarre un vettore `interest_ranking_user_0` con il ranking dal più alto al più basso (cioè assegnando 0 a quello più alto) dei punteggi di interesse verso ciascun prodotto per l'utente con indice 0 (prima riga di `purchases` e `interest`).
- **(6b)** Dal vettore sopra, usando un operatore di confronto, estrarre un vettore booleano `suggestions_for_user_0` di pari forma con valori `True` in corrispondenza dei 20 punteggi d'interesse più alti.
- **(6c)** Generalizzando le soluzioni dei punti sopra, estrarre una matrice `interest_ranking` con i ranking dei punteggi di interesse di ciascun utente (righe) verso ciascun prodotto (colonne) e una matrice `suggestions` di pari forma con valori `True` in corrispondenza dei 20 punteggi d'interesse più alti per ciascun utente.
- **(6d)** Estrarre i nomi dei 20 prodotti suggeriti per l'utente con indice 0.

In [ ]:
# TEST
assert suggestions[0, 0] == False
assert suggestions[0, 170] == True
assert suggestions[1, 400] == True
assert suggestions[1, 570] == False
print("OK")

## Accuratezza dei suggerimenti di acquisto

Come valutare se i suggerimenti ottenuti in questo modo siano azzeccati?

Una possibilità consiste nel verificare **se gli oggetti suggeriti siano stati effettivamente acquistati** in un successivo momento.

Nel file `purchases-2014.csv` è fornita una seconda lista di acquisti aggiornata, che include anche quelli successivi al 2000 per gli stessi insiemi di utenti e prodotti. Possiamo confrontare questi nuovi acquisti con i suggerimenti.

Come abbiamo fatto per il primo file, creiamo una matrice binaria che indichi quali utenti hanno acquistato quali prodotti.

`zeros_like` crea una matrice di zeri di tipo e forma identiche a una data _(esistono anche `empty_like` e `ones_like`)_.

In [ ]:
purchases_updated = np.zeros_like(purchases)

### Esercizio 7: Caricamento dati in matrice

Impostare a 1 gli elementi di `purchases_updated` corrispondenti alle coppie utente-prodotto nel file CSV `purchases-2014.csv`.

- Non usare una struttura dati intermedia (es. l'insieme `purchases_set` usato sopra).
- Riutilizzare gli stessi dizionari `user_indices` e `item_indices` di prima per garantire che righe e colonne delle matrici combacino.

### Esercizio 8: Selezionare solo i nuovi acquisti

La nuova matrice riporta **tutti** gli acquisti, compresi quelli già indicati nella matrice precedente.

Vogliamo una matrice `new_purchases` in cui siano indicati con 1 **solo gli acquisti successivi** all'analisi svolta sopra.

Quale operazione tra le matrici `purchases` e `purchases_updated` ci permette di ottenere tale risultato?

Estrarre la matrice binaria `new_purchases` da tale operazione e convertirla in matrice booleana.

In [ ]:
new_purchases = ...

### Esercizio 9: Quali nuovi acquisti sono stati suggeriti?

Abbiamo ora le matrici booleane `suggestions` con gli acquisti _suggeriti_ e `new_purchases` con i nuovi acquisti _effettivi_.

Da queste possiamo estrarre una matrice `hits` che indichi quali sono i suggerimenti **validi**, quelli a cui dopo l'analisi è corrisposto un acquisto.

Quale operazione tra le due matrici ci fa ottenere tale risultato?

In [ ]:
hits = ...

### Esercizio 10: Quanti utenti hanno ricevuto almeno un suggerimento valido?

- **(10a)** Ottenere un vettore booleano `satisfied_users` che indichi quali utenti hanno ottenuto almeno un suggerimento valido, cioè almeno un `True` nella riga corrispondente di `hits`
- **(10b)** Ricavare dal vettore stesso il numero di tali utenti
- **(10c)** Ricavare dal vettore stesso la percentuale di tali utenti rispetto al numero totale

In [ ]:
satisfied_users = ...

Si dovrebbe ottenere una percentuale del **34,3%** di utenti soddisfatti.

## Esercizio avanzato: Confronto con una selezione casuale di prodotti

_(si propone come esercizio da svolgere opzionalmente dopo la lezione)_

Per valutare quanto il risultato ottenuto sia buono, possiamo misurare cosa otterremmo **suggerendo _N_ prodotti a caso** a ciascun utente.

Con questa procedura, si verifica che si otterrebbe una percentuale di clienti soddisfatti intorno al **13%**, contro il **35%** ottenuto sopra.

Per ottenere risultati riproducibili, impostare un seed per la generazione di numeri casuali tramite la seguente funzione.

In [ ]:
np.random.seed(123)

- **(a)** generare una matrice `random_interest` di forma pari ad `interest` con valori casuali
  - usare una funzione per generare valori casuali in un intervallo continuo, ad es `np.random.random`, indicando la forma desiderata

In [ ]:
random_interest = ...

- **(b)** impostare a 0 i valori della matrice corrispondenti a prodotti già acquistati

- **(c)** selezionare gli _N_ prodotti di maggiore interesse per ogni utente
  - come sopra, generare la matrice con i ranking lungo ciascuna riga, quindi selezionare i ranking appropriati

In [ ]:
random_interest_ranking = ...
random_suggestions = ...

- **(d)** eseguire il confronto con i prodotti effettivamente acquistati ed estrarre il vettore che indica gli utenti con almeno un suggerimento valido

In [ ]:
random_hits = ...
randomly_satisfied_users = ...

- **(e)** calcolare la percentuale di tali utenti rispetto al totale